In [21]:
from datasets import get_dataset_config_names, load_dataset
from datasets import DatasetDict, Dataset, Split

In [28]:
def find_key(data: dict[str, list[str]], query: str) -> str | None:
    return next((k for k, v in data.items() if query in v), None)
SUBSET_MAPPING = {
    "Chat": [
        "alpacaeval-easy",
        "alpacaeval-length",
        "alpacaeval-hard",
        "mt-bench-easy",
        "mt-bench-med",
    ],
    "Chat Hard": [
        "mt-bench-hard",
        "llmbar-natural",
        "llmbar-adver-neighbor",
        "llmbar-adver-GPTInst",
        "llmbar-adver-GPTOut",
        "llmbar-adver-manual",
    ],
    "Safety": [
        "refusals-dangerous",
        "refusals-offensive",
        "xstest-should-refuse",
        "xstest-should-respond",
        "donotanswer",
    ],
    "Reasoning": [
        "math-prm",
        "hep-cpp",
        "hep-go",
        "hep-java",
        "hep-js",
        "hep-python",
        "hep-rust",
    ],
}

In [45]:
import os
os.environ["HF_TOKEN"] = ""
dest = "C4AI-Community/multilingual-reward-bench"
source = "aya-rm-multilingual/multilingual-reward-bench-gtranslate"

configs = get_dataset_config_names(source)
order = ["id", "target_language", "prompt", "chosen", "rejected", "chosen_model", "rejected_model", "subset"]
for config in configs:
    if config == "deu_Latn":
        print(config)
        source_df = load_dataset(source, config)["filtered"].to_pandas()
        source_df = source_df[order].rename(columns={"target_language": "language", "subset": "source"})
        source_df["category"] = source_df["source"].apply(lambda x: find_key(SUBSET_MAPPING, x))
        new_ds = Dataset.from_pandas(source_df, split=Split.TEST)

        new_ds.push_to_hub(dest, config_name=config, split=str(Split.TEST))

deu_Latn


Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


In [46]:
df = load_dataset("aya-rm-multilingual/multilingual-reward-bench-maple", split="all").to_pandas()

Generating filtered split: 800 examples [00:00, 35877.88 examples/s]


In [54]:
df = df.drop(columns="id").reset_index().rename(columns={"index": "id"})

In [57]:
df["subset"] = df["subset"].apply(lambda x: x.replace("maple", "translation"))

In [60]:
ds = Dataset.from_pandas(df, split="translation")

In [62]:
ds.push_to_hub(dest, config_name="translation", split=str(Split.TEST))

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/C4AI-Community/multilingual-reward-bench/commit/711a1a1e051bea0610156df235405e26857263b0', commit_message='Upload dataset', commit_description='', oid='711a1a1e051bea0610156df235405e26857263b0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/C4AI-Community/multilingual-reward-bench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='C4AI-Community/multilingual-reward-bench'), pr_revision=None, pr_num=None)